In [13]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np
from tqdm import tqdm
from sklearn.cluster import KMeans


In [2]:
PROJ_DIM = 25
TOPIC_NUM = 20
MIN_TOPIC_SIZE = 100000
BATCH_SIZE = 256

In [3]:
acm_df = pd.read_csv('/home/student/FinalProject/PaperFeedback/Datasets/acm_citation_network_v8.csv')

In [4]:
acm_df['venue'] = acm_df['venue'].fillna('')
acm_df['venue'] = acm_df['venue'].apply(lambda x: str(x))
acm_df['id'] = range(len(acm_df))

In [5]:
acm_df[['id', 'venue']]

,id,venue
0,0,INFORMS Journal on Computing
1,1,Theoretical Computer Science
2,2,Theoretical Computer Science
3,3,Graphics Interface 1990
4,4,Using program slicing in software maintenance
...,...,...
2381670,2381670,The QCP File Format and Media Types for Speech...
2381671,2381671,Multicast Source Discovery Protocol (MSDP)
2381672,2381672,RTP Control Protocol Extended Reports (RTCP XR)
2381673,2381673,Uniform Resource Identifier (URI) Scheme and A...


In [6]:
class DocDataset(Dataset):
    def __init__(self, df, doc_field):
        self.df = df
        self.doc_field = doc_field

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        return self.df.iloc[index]['id'], self.df.iloc[index][self.doc_field]

In [7]:
doc_dataset=  DocDataset(acm_df, doc_field='venue')
doc_dataloader = DataLoader(doc_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8, pin_memory=True)
if torch.cuda.is_available():
    print('using GPU')
    device = 'cuda'
else:
    print('using CPU')
    device = 'cpu'
model = SentenceTransformer('all-MiniLM-L6-v2')
model = model.to(device)

using GPU


In [8]:
doc_embeddings = []
with torch.no_grad():
    for _, batch in tqdm(doc_dataloader):
        batch_embeddings = model.encode(batch, convert_to_tensor=True, show_progress_bar=False)
        doc_embeddings.extend(batch_embeddings.cpu().numpy())

100%|██████████| 9304/9304 [13:12<00:00, 11.74it/s]


In [9]:
doc_embeddings = np.array(doc_embeddings)

In [14]:
kmeans_model = KMeans(n_clusters=TOPIC_NUM, init='k-means++', max_iter=200, n_init=5, random_state=42, algorithm='elkan')
doc_clusters = kmeans_model.fit_predict(doc_embeddings)



Fitting KMeans:   0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 179, in where
KeyboardInterrupt: 
Fitting KMeans: 100%|██████████| 1/1 [08:25<00:00, 505.94s/it]
